# Рекомендация тарифов для оператора мобильной связи

Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. Оператор хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Цель - построить модель для задачи классификации, которая выберет подходящий тариф.

## Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Оглавление проекта:

1. Откроем и изучим файл
2. Разобьём данные на выборки
3. Исследуем модели
4. Проверим модель на тестовой выборке
5. Проверим модель на адекватность

## 1. Откроем и изучим файл

Импортируем все необходимые библиотеки и инструменты.

In [133]:
import pandas as pd

In [134]:
import joblib

In [135]:
from sklearn.model_selection import train_test_split

In [136]:
from sklearn.metrics import accuracy_score

In [137]:
from sklearn.tree import DecisionTreeClassifier

In [138]:
from sklearn.linear_model import LogisticRegression

In [139]:
from sklearn.ensemble import RandomForestClassifier

In [140]:
users_behavior = pd.read_csv('/datasets/users_behavior.csv')

Ознакомимся с данными.

In [141]:
users_behavior.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [142]:
users_behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Вывод:** Датасет состоит из 3214 наблюдений и 5 столбцов. Предобработка данных уже выполнена, поэтому пропусков нет, а все данные в нужном формате.

## 2. Разобьём данные на выборки

Для корректной проверки будущей модели нам необходимо разбить данные на 3 выборки - обучающую, валидационную и тестовую.

Сначала отделим от датасета 60% наблюдений - это будет обучающая выборка. Оставщиеся данные поделим пополам - на тестовую и валидационную выборки. Таким образом мы разобьем данные в соотношении 60/20/20.

In [143]:
users_behavior_train, users_behavior_rest = train_test_split(users_behavior, test_size=0.4, random_state=666)

In [144]:
users_behavior_test, users_behavior_valid = train_test_split(users_behavior_rest, test_size=0.5, random_state=666)

In [145]:
users_behavior_train.shape

(1928, 5)

In [146]:
users_behavior_test.shape

(643, 5)

In [147]:
users_behavior_valid.shape

(643, 5)

**Вывод:** Разделили данные на три выборки в соотношении 60/20/20 соответственно:
- users_behavior_train - обучающая
- users_behavior_test - тестовая
- users_behavior_valid - валидационная

## 3. Исследуем модели

Подготовим данные и обучим модель, а затем исследуем качество разных моделей, меняя гиперпараметры.

In [148]:
#подготовим признаки и целевой признак обучающей и валидационной выборок
features_train = users_behavior_train.drop(['is_ultra'], axis=1)
target_train = users_behavior_train['is_ultra']

In [149]:
features_valid = users_behavior_valid.drop(['is_ultra'], axis=1)
target_valid = users_behavior_valid['is_ultra']

##### DecisionTreeClassifier

In [150]:
#исследуем работу модели с различными значениями гиперпараметра max_depth
for depth in range(3, 7, 1):
    model1 = DecisionTreeClassifier(random_state=666, max_depth=depth)
    model1.fit(features_train, target_train)
    predictions_train = model1.predict(features_train)
    predictions_valid = model1.predict(features_valid)
    print("Accuracy при max_depth =", depth)
    print("Обучающая выборка:", accuracy_score(target_train, predictions_train))
    print("Валидационная выборка:", accuracy_score(target_valid, predictions_valid))
    print()

Accuracy при max_depth = 3
Обучающая выборка: 0.8018672199170125
Валидационная выборка: 0.8040435458786936

Accuracy при max_depth = 4
Обучающая выборка: 0.8044605809128631
Валидационная выборка: 0.8009331259720062

Accuracy при max_depth = 5
Обучающая выборка: 0.8220954356846473
Валидационная выборка: 0.80248833592535

Accuracy при max_depth = 6
Обучающая выборка: 0.8298755186721992
Валидационная выборка: 0.80248833592535



##### LogisticRegression

In [151]:
model2 = LogisticRegression(random_state=666, solver='lbfgs')

In [152]:
model2.fit(features_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=666, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [153]:
predictions_train2 = model2.predict(features_train)
predictions_valid2 = model2.predict(features_valid)

In [154]:
print("Обучающая выборка:", accuracy_score(target_train, predictions_train2))
print("Валидационная выборка:", accuracy_score(target_valid, predictions_valid2))

Обучающая выборка: 0.754149377593361
Валидационная выборка: 0.7511664074650077


##### RandomForestClassifier

In [155]:
#исследуем работу модели с различными значениями гиперпараметра n_estimators
for estim in range(2, 6, 1):
    model3 = RandomForestClassifier(random_state=666, n_estimators=estim)
    model3.fit(features_train, target_train)
    predictions_train3 = model3.predict(features_train)
    predictions_valid3 = model3.predict(features_valid)
    print("Accuracy при n_estimators =", estim)
    print("Обучающая выборка:", accuracy_score(target_train, predictions_train3))
    print("Валидационная выборка:", accuracy_score(target_valid, predictions_valid3))
    print()

Accuracy при n_estimators = 2
Обучающая выборка: 0.9056016597510373
Валидационная выборка: 0.7807153965785381

Accuracy при n_estimators = 3
Обучающая выборка: 0.9517634854771784
Валидационная выборка: 0.7667185069984448

Accuracy при n_estimators = 4
Обучающая выборка: 0.9481327800829875
Валидационная выборка: 0.7978227060653188

Accuracy при n_estimators = 5
Обучающая выборка: 0.9714730290456431
Валидационная выборка: 0.7931570762052877



**Вывод:** 
- модель случайного леса отработала лучше двух других моделей (хотя и они прошли порог точности в 0.75),
- при увеличении значения max_depth у модели решающего дерева успешность работы модели возрастает, поэтому будем использовать значение 6, 
- в случае с RandomForestClassifier остановимся на значении n_estimators=4, которое дало наилучший результат на валидационной выборке.

## 4. Проверим модели на тестовой выборке

Объедим обучающую и валидационную выборки, а затем снова обучим модель перед тестовой проверкой.

In [156]:
features_merged = (users_behavior_train
                   .drop(['is_ultra'], axis=1)
                   .append([users_behavior_valid
                            .drop(['is_ultra'], axis=1)])
                  )
target_merged = users_behavior_train['is_ultra'].append([users_behavior_valid['is_ultra']])

In [157]:
features_test = users_behavior_test.drop(['is_ultra'], axis=1)
target_test = users_behavior_test['is_ultra']

##### DecisionTreeClassifier

In [158]:
#точность при обучении на объединенной выборке
model1 = DecisionTreeClassifier(random_state=666, max_depth=6)
model1.fit(features_merged, target_merged)
model1.predict(features_test)
model1.score(features_test, target_test)

0.7791601866251944

In [159]:
#точность при обучении только на обучающей выборке
model1 = DecisionTreeClassifier(random_state=666, max_depth=6)
model1.fit(features_train, target_train)
model1.predict(features_test)
model1.score(features_test, target_test)

0.7900466562986003

##### RandomForestClassifier

In [160]:
#точность при обучении на объединенной выборке
model3 = RandomForestClassifier(random_state=666, n_estimators=4)
model3.fit(features_merged, target_merged)
model3.predict(features_test)
model3.score(features_test, target_test)

0.7589424572317263

In [161]:
#точность при обучении только на обучающей выборке
model3 = RandomForestClassifier(random_state=666, n_estimators=4)
model3.fit(features_train, target_train)
model3.predict(features_test)
model3.score(features_test, target_test)

0.7713841368584758

**Вывод:**
- на тестовой выборке модель DecisionTreeClassifier сработала лучше, чем RandomForestClassifier,
- точность моделей падает примерно на 0.02 при обучении на объединенной выборке,
- значенение точности модели DecisionTreeClassifier - **0.79** (при обучении на трейне). Получается, что модель ошибается примерно в двух случаях из десяти.

## 5. Проверим модель на адекватность

Проверим модель на адекватность двумя способами.

**1**

Для проверки адекватности модели сравним точность её предсказания с точностью константы:

In [162]:
CONST = (users_behavior['is_ultra'].value_counts() / users_behavior.shape[0]).loc[0]
if CONST < 0.79:
    print('Модель адекватна')

Модель адекватна


**2**


По другой версии, чтобы оценить адекватность модели в задачах классификации, нужно сравнить её со случайной. 

У нас есть два значения целевого признака `is_ultra` - 1 и 0. Если бы модель предсказывала одно из двух чисел случайным образом, значение ее точности было бы 0.5. С моделью DecisionTreeClassifier мы добились точности 0.79, что гораздо выше.

Следовательно, модель прошла проверку на адекватность.